In [1]:
%matplotlib inline

In [2]:
!hostname

mum-hpc2-gpu5


In [3]:
%cd ICRA_2023

/home/ssap0002/ICRA_2023


In [4]:
from VRNN_DAT import VRNN_DAT
from RNN import GRU, LSTM
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt 

2022-09-14 16:17:16.960734: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


## Data Loading

In [5]:
import os
import numpy as np
import pickle5 as pickle
from sklearn.preprocessing import MinMaxScaler, StandardScaler

ORI_DATA_DIR = 'data/gripper/full'
HALF_DATA_DIR = 'data/gripper/half'
SYNTH_DATA_DIR = 'output/'

In [6]:
def to_timeseries(X, y, time_steps = 50):
    dataX = []
    dataY = []
    # Cut data by sequence length
    for i in range(0, len(X) - time_steps):
        _x = X[i:i+time_steps, :]
        _y = y[i:i+time_steps, :]
        dataX.append(_x)
        dataY.append(_y)
    return np.array(dataX), np.array(dataY)

def revert_timeseries(X, time_steps = 100):
  # if len(X[0]) == time_steps and time_steps != 1:
  #   return np.array(X[0])
  head = X[0]
  tail = np.array([f[time_steps-1] for f in X])
  return np.concatenate((head, tail))

def load_pickle(filepath):
    with open(filepath,'rb') as f:
      x = pickle.load(f)
      return x
    
def _preprocess(data):
  # Features
  flex = data['Flex'][0]
  pressure = data['Pressure'][0]

  # Labels
  f1, f2 = data['Force']
  x1, x2, x3, x4, x5, x6, x7, x8, x9, x10 = data['PosX']
  y1, y2, y3, y4, y5, y6, y7, y8, y9, y10 = data['PosY']
      
  x = list(zip(flex, pressure, f1, f2, x1, x2, x3, x4, x5, x6, x7, x8, x9, x10, y1, y2, y3, y4, y5, y6, y7, y8, y9, y10))
  return np.array(x)

def shuffle(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]

def load_data(s, _from='Train', scaler=None):
  raw_data = load_pickle("data/new_soro_all.pkl")
  train = raw_data.xs(_from, axis=0, level=1, drop_level=True)

  raw_data = _preprocess(train[s])

  # Normalize 
  if scaler is None:
      label_scaler = MinMaxScaler(clip=True)
      feature_scaler = MinMaxScaler(clip=True)
    
      features, labels = feature_scaler.fit_transform(raw_data[:, :2]), label_scaler.fit_transform(raw_data[:, 2:])
      print(f"creating scaler from {s}")
      
  else:
    print("reusing scaler")
    label_scaler, feature_scaler = scaler
    features, labels = feature_scaler.transform(raw_data[:, :2]), label_scaler.transform(raw_data[:, 2:])
    

  # Timeseries
  X_train, y_train = to_timeseries(features, labels, params['timesteps'])

  return  X_train, y_train, (label_scaler, feature_scaler)

In [7]:
def load_dataset(datatype, datasets, batch_size, no_state_data=[], scaler=None):
    X_train, y_train, domain_labels = [], [], []
    
    if datatype == 'test':
        for scenario in datasets:
            _x, _y, scaler = load_data(scenario, _from='Test', scaler=scaler)
            X_train.append(np.array(_x).astype('float32')), y_train.append(np.array(_y).astype('float32'))
            if scenario in no_state_data:
                domain_labels.append([0 for _ in range(len(_x))])
            else:
                domain_labels.append([1 for _ in range(len(_x))])

        for i, (x,y) in enumerate(zip(X_train, y_train)):
            if i == 0:
                dataset = tf.data.Dataset.from_tensor_slices((x, y, domain_labels[i]))
                dataset = dataset.batch(batch_size)
            
            else: 
                _d = tf.data.Dataset.from_tensor_slices((x, y, domain_labels[i]))
                _d = _d.batch(batch_size)
                dataset = dataset.concatenate(_d)
        return dataset, scaler
    
    elif datatype == 'train':
        for scenario in datasets:
            _x, _y, scaler = load_data(scenario, _from='Train', scaler=scaler)
            X_train.append(np.array(_x).astype('float32')), y_train.append(np.array(_y).astype('float32'))
            if scenario in no_state_data:
                domain_labels.append([0 for _ in range(len(_x))])
            else:
                domain_labels.append([1 for _ in range(len(_x))])

        Xdata = np.concatenate(X_train)
        ydata = np.concatenate(y_train)
        Statedata = np.concatenate(domain_labels).astype(np.float32)

        # Shuffle
        p = np.random.permutation(len(Xdata))
        Xdata = Xdata[p]
        ydata = ydata[p]
        Statedata = Statedata[p]

        dataset =  tf.data.Dataset.from_tensor_slices((Xdata ,ydata, Statedata))
        return dataset.batch(batch_size), scaler

In [8]:
def get_dataset(model_type, train_scenario, batch_size, test_source=False):
    
    if train_scenario == "tip_contact":        
        all_datasets = ['osc_tip_30', 'osc_rand_30', 'rand_tip_30', 'rand_rand_60']
    elif train_scenario == "random_contact":
        all_datasets = ['osc_rand_30', 'rand_rand_60','osc_tip_30', 'rand_tip_30']
    elif train_scenario == "osc":        
        all_datasets = ['osc_tip_30', 'osc_rand_30', 'rand_tip_30', 'rand_rand_60']
    elif train_scenario == "rand":
        all_datasets = ['rand_tip_30', 'rand_rand_60', 'osc_tip_30', 'osc_rand_30']
    
        
    if train_scenario == "tip_contact":
        _train_d = ['osc_tip_30', 'rand_tip_30']
    elif train_scenario == "random_contact":
        _train_d = ['osc_rand_30', 'rand_rand_60']
    elif train_scenario == "osc":
        _train_d = ['osc_tip_30', 'osc_rand_30']
    elif train_scenario == "rand":
        _train_d = ['rand_tip_30', 'rand_rand_60']
    
    if test_source:
        _test_d = _train_d
        
    else:
        _test_d = [d for d in all_datasets if d not in _train_d]
    
        
    if model_type == "GRU" or model_type == "LSTM":           
        train_dataset, scaler = load_dataset('train', _train_d, batch_size)
    
    elif "VRNN_DAT" in model_type:
        train_dataset, scaler = load_dataset('train', all_datasets, batch_size, no_state_data=_test_d)
        
    test_dataset, _ = load_dataset('test', _test_d, batch_size, scaler=scaler)
    
    return train_dataset, test_dataset, scaler

def get_model(model_type, params=None):
    if model_type == "GRU":
        return GRU(params['x_dim'], params['timesteps'], params['z_dim'], params['h_dim'])
    elif model_type == "LSTM":
        return LSTM(params['x_dim'], params['timesteps'], params['z_dim'], params['h_dim'])
    elif model_type == "VRNN":
        pass
    elif "VRNN_DAT" in model_type:
        return VRNN_DAT(params['x_dim'], params['h_dim'], params['z_dim'], phi_x_dim=params['phi_x_dim'], d_dim=params['d_dim'], vrnn_lr=params['lr'], adv_lr=params['adv_lr'], clip=params['clip'], rnn=params['rnn'])

def evaluate_model(model, test_dataset):
    pred, y_test = [], []
    for step, (x_batch_test, y_batch_test, _) in enumerate(test_dataset):
        _pred = model.predict(x_batch_test)
        pred.append(_pred)
        y_test.append(y_batch_test)
        
    pred = np.concatenate(pred)
    y_test = np.concatenate(y_test)
    
    # force_x_rmse_metric = tf.keras.metrics.RootMeanSquaredError()
    # force_z_rmse_metric = tf.keras.metrics.RootMeanSquaredError()
    force_rmse_metric = tf.keras.metrics.RootMeanSquaredError()
    marker_rmse_metric = tf.keras.metrics.RootMeanSquaredError()
    rmse_metric =  tf.keras.metrics.RootMeanSquaredError()
    
    # force_x_rmse_metric.update_state(y_test[:, :, 0], pred[:, :, 0]).numpy()
    # force_z_rmse_metric.update_state(y_test[:, :, 1], pred[:, :, 1]).numpy()
    force_rmse_metric.update_state(y_test[:, :, :2], pred[:, :, :2]).numpy()
    force_rmse = force_rmse_metric.result().numpy()

    marker_rmse_metric.update_state(y_test[:, :, 2:], pred[:, :, 2:]).numpy()
    marker_rmse = marker_rmse_metric.result().numpy()
    
    rmse_metric.update_state(y_test, pred).numpy()
    rmse =  rmse_metric.result().numpy()
    
    return rmse, force_rmse, marker_rmse

## Hyperparameters

In [9]:
x_dim = 2 # flex + pressure
h_dim = 128
z_dim = 22 # 2 force + 10*2D coordinates
phi_x_dim = 32
d_dim = 128
n_epochs = 50
clip = 5
lr = 1e-3
adv_lr = 1e-3
batch_size = 128 # 32
seed = 261199
timesteps=100
rnn="GRU"
# tf.random.set_seed(seed)

In [10]:
from sorcery import dict_of

params = dict_of(x_dim, h_dim, z_dim, phi_x_dim, timesteps, d_dim, n_epochs, clip, lr, adv_lr, batch_size, rnn)

## Verify

In [11]:
import pandas as pd
model_types = ["VRNN_DAT"]
# model_types = ["LSTM"]
df = pd.DataFrame(columns=["Model", "Train Scenario", "Force_X_RMSE", "Force_Z_RMSE", "Marker_RMSE", "Run"])
                  
def verify(model_type, train_scenario, df, run, test_source=False): 
    if "GRU" in model_type:
        params['rnn'] = 'GRU'
    elif "LSTM" in model_type:
        params['rnn'] = 'LSTM'
        
    print(f"================={model_type}_{params['rnn']}:{train_scenario}:{run}==================")
    # Prepare dataset and model  
    if test_source:
        train_dataset, test_dataset, scalers = get_dataset(model_type, train_scenario, params['batch_size'], test_source=True)
    else:
        train_dataset, test_dataset, scalers = get_dataset(model_type, train_scenario, params['batch_size'])
        

    model = get_model(model_type, params)

    # Train
    if model_type == "GRU" or model_type == "LSTM":
        model.compile(loss=tf.losses.MeanSquaredError(), optimizer=tf.optimizers.Adam(), metrics=[tf.metrics.RootMeanSquaredError()])
    else:
        model.compile()
    
    try:
        # Lload weights
        model.load_weights(f"weights/one_scaler/dat_z/{model_type}/{train_scenario}/run_{run}/")

        # Evaluate
        rmse, force_rmse, marker_rmse = evaluate_model(model, test_dataset)


        # Write to df
        df = df.append({"Model": model_type, "Train Scenario": f'{train_scenario}', 'RMSE': rmse, 'Force_RMSE': force_rmse, 'Marker_RMSE': marker_rmse, "Run": run}, ignore_index=True)
    except:
        print(f"No weights for {model_type}/{train_scenario}/run_{run}")
    finally:
        return df

## Test on target

In [12]:
df = pd.DataFrame(columns=["Model", "Train Scenario", "Force_RMSE", "Marker_RMSE", "RMSE", "Run"])

for model_type in ['VRNN_DAT_LSTM', 'VRNN_DAT_GRU', "LSTM", "GRU"]:
    for train_scenario in ["tip_contact", "random_contact", "osc", "rand"]:
    # for train_scenario in ["tip_contact", "random_contact"]:
        for run in range(1,6):
            df = verify(model_type, train_scenario, df, run, test_source=False)

=================VRNN_DAT_LSTM_LSTM:tip_contact:1==================
creating scaler from osc_tip_30
reusing scaler
reusing scaler
reusing scaler


2022-09-14 16:17:21.877141: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-14 16:17:22.281255: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 79132 MB memory:  -> device: 0, name: NVIDIA A100 80GB PCIe, pci bus id: 0000:17:00.0, compute capability: 8.0


reusing scaler
reusing scaler
DAT on Zt
Using LSTM Backend


2022-09-14 16:17:24.266155: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2022-09-14 16:17:24.532504: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8401
/tmp/ipykernel_3482302/2055089692.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({"Model": model_type, "Train Scenario": f'{train_scenario}', 'RMSE': rmse, 'Force_RMSE': force_rmse, 'Marker_RMSE': marker_rmse, "Run": run}, ignore_index=True)


=================VRNN_DAT_LSTM_LSTM:tip_contact:2==================
creating scaler from osc_tip_30
reusing scaler
reusing scaler
reusing scaler
reusing scaler
reusing scaler
DAT on Zt
Using LSTM Backend


...

3/3 [==============================] - 0s 2ms/step
=================GRU_GRU:rand:4==================
creating scaler from rand_tip_30


/tmp/ipykernel_3482302/2055089692.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({"Model": model_type, "Train Scenario": f'{train_scenario}', 'RMSE': rmse, 'Force_RMSE': force_rmse, 'Marker_RMSE': marker_rmse, "Run": run}, ignore_index=True)


reusing scaler
reusing scaler
reusing scaler
3/3 [==============================] - 0s 2ms/step
=================GRU_GRU:rand:5==================
creating scaler from rand_tip_30


/tmp/ipykernel_3482302/2055089692.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({"Model": model_type, "Train Scenario": f'{train_scenario}', 'RMSE': rmse, 'Force_RMSE': force_rmse, 'Marker_RMSE': marker_rmse, "Run": run}, ignore_index=True)


reusing scaler
reusing scaler
reusing scaler
No weights for GRU/rand/run_5


In [13]:
df.head()

,Model,Train Scenario,Force_RMSE,Marker_RMSE,RMSE,Run
0,VRNN_DAT_LSTM,tip_contact,0.069380,0.100573,0.098148,1
1,VRNN_DAT_LSTM,tip_contact,0.066733,0.094179,0.092022,2
2,VRNN_DAT_LSTM,tip_contact,0.068783,0.099825,0.097412,3
3,VRNN_DAT_LSTM,tip_contact,0.084046,0.095659,0.094663,4
4,VRNN_DAT_LSTM,random_contact,0.034815,0.070605,0.068132,1


In [14]:
df[["Model", "Train Scenario", "RMSE"]].groupby(["Model", "Train Scenario"]).describe()

RMSE                                          \
                             count      mean       std       min       25%   
Model         Train Scenario                                                 
GRU           osc              4.0  0.157737  0.004048  0.153707  0.154840   
              rand             4.0  0.148652  0.002003  0.146387  0.147535   
LSTM          osc              4.0  0.156353  0.002849  0.153636  0.154757   
              rand             4.0  0.152177  0.002194  0.149928  0.151100   
              random_contact   5.0  0.108745  0.002634  0.106734  0.107247   
              tip_contact      5.0  0.267792  0.007404  0.258879  0.263929   
VRNN_DAT_GRU  osc              5.0  0.100877  0.001701  0.098606  0.099967   
              rand             5.0  0.102074  0.004438  0.098039  0.099607   
              random_contact   4.0  0.070393  0.004345  0.065085  0.067757   
              tip_contact      3.0  0.095361  0.002588  0.093754  0.093868   
VRNN_DAT_LSTM osc              5.0  0.100506  0.001397  0.098641  0.100150   
              rand             5.0  0.105810  0.003805  0.100114  0.104895   
              random_contact   5.0  0.068473  0.002519  0.066022  0.066080   
              tip_contact      4.0  0.095561  0.002796  0.092022  0.094003   

                                                            
                                   50%       75%       max  
Model         Train Scenario                                
GRU           osc             0.157314  0.160212  0.162612  
              rand            0.148552  0.149670  0.151118  
LSTM          osc             0.155745  0.157341  0.160286  
              rand            0.151808  0.152885  0.155162  
              random_contact  0.107377  0.109222  0.113146  
              tip_contact     0.265235  0.274793  0.276122  
VRNN_DAT_GRU  osc             0.101024  0.101695  0.103091  
              rand            0.100099  0.103432  0.109193  
              random_contact  0.071022  0.073657  0.074441  
              tip_contact     0.093982  0.096164  0.098346  
VRNN_DAT_LSTM osc             0.100380  0.100835  0.102525  
              rand            0.105851  0.107859  0.110331  
              random_contact  0.068132  0.070860  0.071273  
              tip_contact     0.096037  0.097596  0.098148

## Test on source

In [15]:
df = pd.DataFrame(columns=["Model", "Train Scenario", "Force_RMSE", "Marker_RMSE", "RMSE", "Run"])

for model_type in ['VRNN_DAT_LSTM', 'VRNN_DAT_GRU', "LSTM", "GRU"]:
    for train_scenario in ["tip_contact", "random_contact", "osc", "rand"]:
    # for train_scenario in ["tip_contact", "random_contact"]:
    
        for run in range(1,6):
            df = verify(model_type, train_scenario, df, run, test_source=True)

=================VRNN_DAT_LSTM_LSTM:tip_contact:1==================
creating scaler from osc_tip_30
reusing scaler
reusing scaler
reusing scaler
reusing scaler
reusing scaler
DAT on Zt
Using LSTM Backend


/tmp/ipykernel_3482302/2055089692.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({"Model": model_type, "Train Scenario": f'{train_scenario}', 'RMSE': rmse, 'Force_RMSE': force_rmse, 'Marker_RMSE': marker_rmse, "Run": run}, ignore_index=True)


=================VRNN_DAT_LSTM_LSTM:tip_contact:2==================
creating scaler from osc_tip_30
reusing scaler
reusing scaler
reusing scaler
reusing scaler
reusing scaler
DAT on Zt
Using LSTM Backend


/tmp/ipykernel_3482302/2055089692.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({"Model": model_type, "Train Scenario": f'{train_scenario}', 'RMSE': rmse, 'Force_RMSE': force_rmse, 'Marker_RMSE': marker_rmse, "Run": run}, ignore_index=True)


=================VRNN_DAT_LSTM_LSTM:tip_contact:3==================
creating scaler from osc_tip_30
reusing scaler
reusing scaler
reusing scaler
reusing scaler
reusing scaler
DAT on Zt
Using LSTM Backend


...

creating scaler from rand_tip_30
reusing scaler
reusing scaler
reusing scaler
1/1 [==============================] - 0s 24ms/step
=================GRU_GRU:rand:4==================


/tmp/ipykernel_3482302/2055089692.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({"Model": model_type, "Train Scenario": f'{train_scenario}', 'RMSE': rmse, 'Force_RMSE': force_rmse, 'Marker_RMSE': marker_rmse, "Run": run}, ignore_index=True)


creating scaler from rand_tip_30
reusing scaler
reusing scaler
reusing scaler
1/1 [==============================] - 0s 24ms/step
=================GRU_GRU:rand:5==================


/tmp/ipykernel_3482302/2055089692.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({"Model": model_type, "Train Scenario": f'{train_scenario}', 'RMSE': rmse, 'Force_RMSE': force_rmse, 'Marker_RMSE': marker_rmse, "Run": run}, ignore_index=True)


creating scaler from rand_tip_30
reusing scaler
reusing scaler
reusing scaler
No weights for GRU/rand/run_5


In [16]:
df.head(10)

,Model,Train Scenario,Force_RMSE,Marker_RMSE,RMSE,Run
0,VRNN_DAT_LSTM,tip_contact,0.070883,0.129765,0.125558,1
1,VRNN_DAT_LSTM,tip_contact,0.069050,0.126442,0.122342,2
2,VRNN_DAT_LSTM,tip_contact,0.067601,0.127151,0.122935,3
3,VRNN_DAT_LSTM,tip_contact,0.089320,0.126193,0.123297,4
4,VRNN_DAT_LSTM,random_contact,0.034906,0.057882,0.056183,1
5,VRNN_DAT_LSTM,random_contact,0.034490,0.057058,0.055388,2
6,VRNN_DAT_LSTM,random_contact,0.034464,0.059247,0.057438,3
7,VRNN_DAT_LSTM,random_contact,0.033539,0.060435,0.058503,4
8,VRNN_DAT_LSTM,random_contact,0.034134,0.057303,0.055597,5
9,VRNN_DAT_LSTM,osc,0.079293,0.123074,0.119757,1


In [17]:
df[["Model", "Train Scenario", "RMSE"]].groupby(["Model", "Train Scenario"]).describe()

RMSE                                          \
                             count      mean       std       min       25%   
Model         Train Scenario                                                 
GRU           osc              4.0  0.091456  0.003130  0.088000  0.089215   
              rand             4.0  0.088044  0.002209  0.086099  0.086232   
              random_contact   5.0  0.044622  0.000462  0.043886  0.044446   
              tip_contact      5.0  0.093605  0.003048  0.089867  0.091324   
LSTM          osc              4.0  0.093859  0.003198  0.091268  0.091602   
              rand             4.0  0.088266  0.000913  0.087627  0.087781   
              random_contact   5.0  0.044769  0.001385  0.042902  0.044343   
              tip_contact      5.0  0.095177  0.002406  0.092356  0.094127   
VRNN_DAT_GRU  osc              5.0  0.115720  0.006890  0.108070  0.112466   
              rand             5.0  0.089550  0.002613  0.086535  0.087164   
              random_contact   4.0  0.055866  0.001875  0.053296  0.055303   
              tip_contact      3.0  0.117875  0.000916  0.117054  0.117381   
VRNN_DAT_LSTM osc              5.0  0.118669  0.002079  0.116258  0.117716   
              rand             5.0  0.092468  0.004579  0.087970  0.089805   
              random_contact   5.0  0.056622  0.001320  0.055388  0.055597   
              tip_contact      4.0  0.123533  0.001406  0.122342  0.122787   

                                                            
                                   50%       75%       max  
Model         Train Scenario                                
GRU           osc             0.091764  0.094005  0.094294  
              rand            0.087767  0.089579  0.090545  
              random_contact  0.044897  0.044905  0.044977  
              tip_contact     0.093776  0.095879  0.097181  
LSTM          osc             0.092960  0.095217  0.098246  
              rand            0.087911  0.088396  0.089617  
              random_contact  0.044500  0.045474  0.046623  
              tip_contact     0.094292  0.096554  0.098558  
VRNN_DAT_GRU  osc             0.112853  0.119629  0.125583  
              rand            0.090094  0.091516  0.092442  
              random_contact  0.056202  0.056764  0.057766  
              tip_contact     0.117708  0.118285  0.118862  
VRNN_DAT_LSTM osc             0.117965  0.119757  0.121650  
              rand            0.091306  0.093426  0.099835  
              random_contact  0.056183  0.057438  0.058503  
              tip_contact     0.123116  0.123863  0.125558